In [286]:
# importing all packages that may be required
import os
#import gc
import re
import csv
import html
import copy
from tqdm import tqdm
tqdm.pandas()
import time
import mailbox
import unicodedata
import pandas as pd
#import multiprocessing
from bs4 import BeautifulSoup
from dask import dataframe as dd
from dask.multiprocessing import get
import nltk
from dask.diagnostics import ProgressBar
from nltk.stem import WordNetLemmatizer as wnl
from nltk import pos_tag as pt
from nltk.corpus import wordnet as wn

In [94]:
cwd = os.getcwd()

In [95]:
# importing mailbox
mail_container = mailbox.mbox(cwd+'/raw_data/podesta-emails.mbox-2016-11-06')

In [113]:
# making working copy
mail_cp = copy.copy(mail_container)

In [243]:
def clean(msg_content):
    if msg_content!=None:
        msg_clean = "".join([ch for ch in str(msg_content) if ord(ch)<= 128])
        for r in ['\\n', '\n', '\t', '\\t', 'None', 'none']: msg_clean = msg_clean.replace(r, ' ')
        msg_clean = re.sub(r' +', ' ', msg_clean)
    return msg_clean

In [245]:
def get_wordnet_pos(word):
    if word.startswith('J'):
        return wn.ADJ
    elif word.startswith('V'):
        return wn.VERB
    elif word.startswith('N'):
        return wn.NOUN
    elif word.startswith('R'):
        return wn.ADV
    else:
        return wn.NOUN

def lemat(msg_clean):
    l=[]
    msg_clean = nltk.word_tokenize(msg_clean)
    ptag = pt(msg_clean)
    for word in ptag:
        lem = wnl().lemmatize(word[0], get_wordnet_pos(word[1]))
        if lem!=None or lem!='' or lem!=' ':
            l.append(str(lem.lower()))
        else:
            pass
    return ' '.join(w for w in l)

In [247]:
def extract(mail_cp):
    try:
        with open('podesta_mailbox_1.csv', 'w', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['date', 'from', 'to', 'subject', 'originator_org', 'spam_header', 'msg_body'])
            for mail in mail_cp.itervalues():
                if mail.is_multipart():
                    for part in mail.walk():
                        if part.get_content_type()=='text/plain':
                            msg_content = part.get_payload(decode=True)
                else:
                    if mail.get_content_type()=='text/plain':
                        msg_content = mail.get_payload(decode=True)
                writer.writerow([mail['Date'], mail['From'], mail['To'],
                                 mail['Subject'], mail['X-OriginatorOrg'],
                                 mail['SpamDiagnosticMetadata'],clean(msg_content.lower())])
    except:
        pass

In [248]:
# running function and display time
%time extract(mail_cp)
# 15 mins - 9.18 gb

CPU times: user 22min 38s, sys: 13.7 s, total: 22min 52s
Wall time: 23min 19s


In [ ]:
# deleting unwanted variables
%who
del extract
del mail_cp
del mailbox
del mail_container

In [ ]:
# spam headers = https://docs.microsoft.com/en-us/office365/securitycompliance/anti-spam-message-headers

In [249]:
# listing datatypes of dataframe and including in importing csv with 50Mb blocksize
dtypes={'from': 'object', 'to': 'object', 'subject': 'object', 'originator_org': 'object', 'spam_header': 'object','msg_body': 'object'}
podesta = pd.read_csv(cwd+'/podesta_mailbox.csv', dtype=dtypes, parse_dates=True)

In [250]:
podesta.head(2)

,date,from,to,subject,originator_org,spam_header,msg_body
0,"Fri, 3 Apr 2015 14:06:30 +0000",Tina Flournoy <Tina@presidentclinton.com>,John Podesta <john.podesta@gmail.com>,just talked to him,NaN,NaN,b' sc was the first thing he mentioned. told h...
1,"Mon, 22 Feb 2016 23:00:30 +0000",Ira Shapiro <ira@shapiroglobal.com>,John Podesta <john.podesta@gmail.com>,today's fundraiser,shapiroglobal.com,NSPM,"b""john, sorry we didn't get to visit this morn..."


In [251]:
def mail_ext(x):
    x = re.search(r'\S+@\S+', str(x).lower())
    if x:
        return (x.group().replace('>','').replace('<','').lower())

def name_r(em):
    em = str(em).lower()
    t = re.sub(r'\S+@\S+', '', em)
    if  t==None or t==" " or t=="":
        return re.search(r'.*?(?=@)', em).group().replace('>','').replace('<','').replace('.',' ')
    else:
        return re.sub(r'\S+@\S+', '', em)    

In [252]:
podesta['to_email'] = podesta.to.apply(lambda x: mail_ext(x))
podesta['from_email'] = podesta['from'].apply(lambda x: mail_ext(x))

podesta['to_name'] = podesta.to.apply(lambda x: name_r(x))
podesta['from_name'] = podesta['from'].apply(lambda x: name_r(x))


In [253]:
podesta.head(2)

,date,from,to,subject,originator_org,spam_header,msg_body,to_email,from_email,to_name,from_name
0,"Fri, 3 Apr 2015 14:06:30 +0000",Tina Flournoy <Tina@presidentclinton.com>,John Podesta <john.podesta@gmail.com>,just talked to him,NaN,NaN,b' sc was the first thing he mentioned. told h...,john.podesta@gmail.com,tina@presidentclinton.com,john podesta,tina flournoy
1,"Mon, 22 Feb 2016 23:00:30 +0000",Ira Shapiro <ira@shapiroglobal.com>,John Podesta <john.podesta@gmail.com>,today's fundraiser,shapiroglobal.com,NSPM,"b""john, sorry we didn't get to visit this morn...",john.podesta@gmail.com,ira@shapiroglobal.com,john podesta,ira shapiro


In [271]:
%time podesta['lem_body']=podesta.msg_body.progress_apply(lambda x: lemat(x))

In [270]:
#podesta_dd = dd.from_pandas(podesta, npartitions=4)

In [285]:
#podesta_dd

In [284]:
#%time pod_lemat = podesta_dd.apply(lambda df: lemat(df.msg_body), meta=pd.Series(dtype='object', name='lemat_body'), axis=1)

#pod_lemat.head(2).compute()

#with ProgressBar():
#    podesta['lemat_body'] = pod_lemat.compute(scheduler='processes')

In [275]:
# podesta['lenat_body']=podesta_dd.map_partitions(lambda df: df.msg_body.apply(lemat), meta=pd.Series(dtype='object', name='lemat_body'))


In [237]:
% who

BeautifulSoup	 clean	 copy	 csv	 cwd	 dd	 dtypes	 e	 em	 
extract	 get	 html	 mail	 mail_container	 mail_cp	 mail_ext	 mailbox	 msg_content	 
name_r	 nltk	 os	 part	 pd	 podesta	 pt	 re	 time	 
unicodedata	 wn	 wnl	 
